In [1]:
%%capture
!pip install pandas scikit-learn transformers sentence-transformers matplotlib umap

# Reconstruction 

We test if from the probes we are able to reconstruct. We test for both the linear and non linear ones. 

1. We get an embedding for a sentence 
2. Get the probe for each token. We add them together.
3. We mean summarize them
4. Compare that to the base representation using R^2





In [4]:
import pickle

In [6]:
!ls

dataset.pkl		       output_log_prev.txt
dict_probes.pkl		       probe_factorization.ipynb
matthieu_test.ipynb	       probing_word_level.ipynb
matthieunlp		       requirements.txt
mlp_classifier.joblib	       train_models.py
multiobject_world_model.ipynb  trained_models
one_hot_encoder.joblib	       vikram_probing_word_level.ipynb
output_log.txt


In [7]:
with open('dataset.pkl', 'rb') as f:
    df = pickle.load(f)

OSError: [Errno 14] Bad address

In [68]:
df.head

,word,embedding,pos,dep,position,sentence_id,sentence
0,The,"[tensor(-0.2899), tensor(0.2611), tensor(-0.30...",DET,det,0,0,The Fulton County Grand Jury said Friday an in...
1,Fulton,"[tensor(-0.1788), tensor(0.6752), tensor(-0.56...",PROPN,compound,1,0,The Fulton County Grand Jury said Friday an in...
2,County,"[tensor(-0.2251), tensor(-0.0033), tensor(-0.2...",PROPN,compound,2,0,The Fulton County Grand Jury said Friday an in...
3,Grand,"[tensor(-0.5069), tensor(0.2878), tensor(0.179...",ADJ,amod,3,0,The Fulton County Grand Jury said Friday an in...
4,Jury,"[tensor(0.2416), tensor(0.0862), tensor(0.1484...",PROPN,nsubj,4,0,The Fulton County Grand Jury said Friday an in...
...,...,...,...,...,...,...,...
435043,outside,"[tensor(0.6717), tensor(0.2565), tensor(0.3406...",ADJ,amod,26,19999,"Of course, most toilets are Eastern style -- a..."
435044,toilets,"[tensor(0.5431), tensor(-0.2458), tensor(-0.18...",NOUN,conj,27,19999,"Of course, most toilets are Eastern style -- a..."
435045,--,"[tensor(0.3646), tensor(0.1068), tensor(0.0683...",PUNCT,punct,28,19999,"Of course, most toilets are Eastern style -- a..."
435046,inside,"[tensor(0.6022), tensor(-0.3015), tensor(-0.66...",ADV,advmod,29,19999,"Of course, most toilets are Eastern style -- a..."


# Reconstructing the Token from the Main probes 

In [18]:
import torch.nn as nn

class LinearProbe(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

class NonlinearProbe(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)
        
class AdaptiveSoftmaxProbe(nn.Module):
    def __init__(self, input_dim, n_classes):
        super().__init__()
        cutoffs = [min(1000, n_classes - 1)]
        if n_classes > 10000:
            cutoffs.append(min(10000, n_classes - 2))
        self.adaptive_softmax = nn.AdaptiveLogSoftmaxWithLoss(
            in_features=input_dim,
            n_classes=n_classes,
            cutoffs=cutoffs,
            div_value=4.0
        )

    def forward(self, x, target=None):
        if target is not None:
            return self.adaptive_softmax(x, target)
        else:
            return self.adaptive_softmax.log_prob(x)

In [34]:
PROBE_DIR = 'trained_models/20250329_183025'

with open('trained_models/20250329_183025/dep_model.pkl', 'rb') as file:
    dep_probe = pickle.load(file)

with open('trained_models/20250329_233721/pos_model.pkl', 'rb') as file:
    pos_probe = pickle.load(file)

with open('trained_models/20250331_022918//word_model.pkl', 'rb') as file:
    word_probe = pickle.load(file)

with open('trained_models/20250329_183025/dep_nonlinear.pkl', 'rb') as file:
    dep_nl_probe = pickle.load(file)

with open('trained_models//20250329_183025/pos_nonlinear.pkl', 'rb') as file:
    pos_nl_probe = pickle.load(file)



In [16]:
weights = word_model.linear.weight

In [55]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# Test if the tokens sum to the full thing 

In [67]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModel

# === Device & Model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

sentence_groups = list(df.groupby("sentence_id"))[:1000]  # limit to first 1000

results = []

for sentence_id, group in tqdm(sentence_groups, desc="Processing"):
    token_embeddings = torch.stack(group['embedding'].tolist())  # shape: (num_tokens, dim)
    sentence_text = group['sentence'].iloc[0]

    mean_embedding = token_embeddings.mean(dim=0)

    inputs = tokenizer(sentence_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        """ 
        output = model(**inputs)
        token_embeddings = output.last_hidden_state[0]  # shape: [seq_len, hidden_dim]
        attention_mask = inputs['attention_mask'][0].unsqueeze(-1)  # shape: [seq_len, 1]
        masked_embeddings = token_embeddings * attention_mask  # zero out [PAD] tokens
        mean_embedding = masked_embeddings.sum(dim=0) / attention_mask.sum()
        """
        
        mean_embedding = token_embeddings.mean()
        minilm_embedding = mean_embedding.to(device)

    mean_np = mean_embedding.cpu().numpy()
    minilm_np = minilm_embedding.cpu().numpy()
    
    cos_sim = cosine_similarity(mean_np.reshape(1, -1), minilm_np.reshape(1, -1))[0, 0]

    print(mean_np)
    print(minilm_np)
    mse = mean_squared_error(mean_np, minilm_np)

    results.append({
        "sentence_id": sentence_id,
        "sentence": sentence_text,
        "cosine_similarity": cos_sim,
        "mse": mse
    })

# === Results & Summary
results_df = pd.DataFrame(results)
mean_cosine = results_df['cosine_similarity'].mean()
mean_mse = results_df['mse'].mean()

print(f"\nAverage Cosine Similarity: {mean_cosine:.4f}")
print(f"Average MSE: {mean_mse:.4f}")


Processing:   0%|          | 0/1000 [00:00<?, ?it/s]

0.00028937435
0.00028937435


TypeError: Input should have at least 1 dimension i.e. satisfy `len(x.shape) > 0`, got scalar `array(0.00028937, dtype=float32)` instead.

# We can now test 

1. For a given sentence 
2. Get the full sentence embedding 


In [ ]:
import pandas as pd
import torch

pos_weights = pos_model.linear.weight.detach().cpu().numpy()
dep_weights = dep_model.linear.weight.detach().cpu().numpy()

def get_probe_weights_for_word(word, tokenizer):
    word_tokens = tokenizer.encode(word)
    
    if len(word_tokens) == 0:
        raise ValueError(f"Word '{word}' not found in the tokenizer's vocabulary.")
    
    word_idx = list(le_word.classes_).index(word)
    probe_weights = word_model.linear.weight[word_idx].detach().cpu().numpy()
    return probe_weights / len(word_tokens)


def compute_sentence_embeddings(df, tokenizer):
    sentence_embeddings = []
    
    for sentence_id, group in df.groupby('sentence_id'):
        weighted_sum = torch.zeros(384)
        total_num_tokens = 0

        for _, row in group.iterrows():
            word = row['word'].lower()
            total_num_tokens += 1
            probe_weight = get_probe_weights_for_word(word, tokenizer)
            weighted_sum += probe_weight

        # get POS
        pos = row['pos']
        pos_idx = list(le_pos.classes_).index(pos)
        pos_probe_weight = pos_weights[pos_idx]

        # get DEP
        dep = row['dep']
        dep_idx = list(le_dep.classes_).index(dep)
        dep_probe_weight = dep_weights[dep_idx]

        full_probe_embedding = (weighted_sum.numpy() / total_num_tokens) + pos_probe_weight + dep_probe_weight
     
        sentence_embeddings.append({
            'sentence_id': sentence_id,
            'sentence': group['sentence'].iloc[0],
            'probe_embedding': full_probe_embedding
        })
    
    return pd.DataFrame(sentence_embeddings)

summed_embeddings = compute_sentence_embeddings(sample_df, tokenizer)
summed_embeddings = summed_embeddings.drop_duplicates('sentence')

In [ ]:
all_sentences = summed_embeddings['sentence'].values.tolist()
inputs = tokenizer(all_sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state

input_ids = inputs['input_ids']
mask = (input_ids != tokenizer.pad_token_id) & (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
masked_embeddings = embeddings[mask]
sentence_embeddings = torch.mean(embeddings, dim=1)